In [1]:
'''
Notebook for FIG2A - produce an illustration of bifurcation analysis for the Punisher
'''
# By Kirill Sechkar

# PACKAGE IMPORTS ------------------------------------------------------------------------------------------------------
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
import pickle
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, transform as bktransform
from math import pi
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS -----------------------------------------------------------------------------------------------------
import synthetic_circuits as circuits
from cell_model import *
from get_steady_state import *
from design_guidance_tools import *

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
 # INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,\
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_b_initialise,
    circuits.punisher_b_ode,
    circuits.punisher_b_F_calc,
    circuits.punisher_b_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [3]:
# SPECIFY THE CIRCUIT'S PARAMETERS
# For the purposes of illustration, we use different parameters to those of the circuit in other simulations

# burdensome gene
par['c_b'] = 100.0
par['a_b'] = 2000.0

# punisher
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 100.0  # promoter strength (unitless)
par['c_int'] = 10.0  # gene concentration (nM)
par['a_int'] = 60.0  # promoter strength (unitless)
par['d_int'] = 6.0  # integrase protein degradation rate (to avoid unnecessary punishment)
par['c_cat'] = 10.0  # gene concentration (nM)
par['a_cat'] = 500.0  # promoter strength (unitless)

# punisher's transcription regulation function
par['K_switch'] = 500.0  # 350.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 3  # 2

# Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.15  # 0.01  # Baseline value of the switch gene's transcription activation function
par['p_switch_ac_frac'] = 0.45  # 0.75  # active fraction of protein (i.e. share of molecules bound by the inducer)

# culture medium
init_conds['s'] = 0.3
par['h_ext'] = 10.5 * (10.0 ** 3)

In [4]:
# get the cellular variables in steady state without burden
e, F_r, h, xis, chis = values_for_analytical(par, ode_with_circuit, init_conds,
                                             circuit_genes, circuit_miscs,
                                             circuit_name2pos,
                                             circuit_F_calc,
                                             circuit_eff_m_het_div_k_het)
# record the cellular variables
cellvars = {'e': e, 'F_r': F_r, # translation elongation rate and ribosome trnscription regulation
            'h': h, # intacellular chlorampenicol concentration
            # burden values
            'xi_a': xis['a'], 'xi_r': xis['r'], 'xi_other_genes': xis['other'], 'xi_cat': xis['cat'],
            'xi_switch_max': xis['switch (max)'], 'xi_int_max': xis['int (max)'], 'xi_prot': xis['prot'],
            # protein degradation correction factors for the switch protein and the integrase
            'chi_switch': chis['switch'], 'chi_int': chis['int']}

xi_all = cellvars['xi_a'] + cellvars['xi_r'] + cellvars['xi_cat'] + cellvars['xi_prot'] + cellvars['xi_other_genes']
xi_mutated = xi_all - cellvars['xi_other_genes']

In [5]:
# FIND THE THRESHOLD BIFURCATION
thresholds_gfchanges_vector=np.array(threshold_mfchanges(par, cellvars))

xi_threshold = thresholds_gfchanges_vector[1]
gfchange_intact = thresholds_gfchanges_vector[4]

In [6]:
# FIND REQUIRED AND REAL F_SWITCH VALUES FOR DIFFERENT VALUES OF XI

# on the x axis, switch protein concentrations
p_switch_sup = pswitch_upper_bound_4nonsaddle(xi_threshold, par,cellvars)
p_switch_axis=np.linspace(0,1.2*p_switch_sup,10000)

# find real F_switch values
F_reals = F_real_calc(p_switch_axis, par)

# find required F_switch values
F_reqs_all = F_req_calc(p_switch_axis, xi_all,par,cellvars)  # all synthetic genes present
F_reqs_threshold = F_req_calc(p_switch_axis, xi_threshold,par,cellvars)  # threshold burden
F_reqs_mutated = F_req_calc(p_switch_axis, xi_mutated,par,cellvars)  # only native genes and CAT

In [7]:
# FIND FIXED POINTS FOR ALL BURDEN VALUES 

# find for all synthetic genes present and just native and cat genes present
findfps_func= lambda p_switch,xi: (F_real_calc(p_switch,par)-F_req_calc(p_switch,xi,par,cellvars))**2
# just do it approximately by detecting minima in suqared difference of real and requiredF_switch values
# for all genes
fps_all = []
findfpfunc_before = findfps_func(p_switch_axis[0],xi_all)
findfpfunc = findfps_func(p_switch_axis[1],xi_all)
for i in range(1,len(p_switch_axis)-1):
    findfpfunc_after = findfps_func(p_switch_axis[i+1],xi_all)
    if(findfpfunc_before>findfpfunc and findfpfunc_after>findfpfunc):
        fps_all.append([p_switch_axis[i],F_reals[i]])
    findfpfunc_before = findfpfunc
    findfpfunc = findfpfunc_after
# for threshold burden
fps_threshold = []
findfpfunc_before = findfps_func(p_switch_axis[0],xi_threshold)
findfpfunc = findfps_func(p_switch_axis[1],xi_threshold)
for i in range(1,len(p_switch_axis)-1):
    findfpfunc_after = findfps_func(p_switch_axis[i+1],xi_threshold)
    if(findfpfunc_before>findfpfunc and findfpfunc_after>findfpfunc):
        fps_threshold.append([p_switch_axis[i],F_reals[i]])
    findfpfunc_before = findfpfunc
    findfpfunc = findfpfunc_after
# for just native and cat genes
fps_mutated = []
findfpfunc_before = findfps_func(p_switch_axis[0],xi_mutated)
findfpfunc = findfps_func(p_switch_axis[1],xi_mutated)
for i in range(1,len(p_switch_axis)-1):
    findfpfunc_after = findfps_func(p_switch_axis[i+1],xi_mutated)
    if(findfpfunc_before>findfpfunc and findfpfunc_after>findfpfunc):
        fps_mutated.append([p_switch_axis[i],F_reals[i]])
    findfpfunc_before = findfpfunc
    findfpfunc = findfpfunc_after
fps_mutated=fps_mutated[1:]  # remove the first one, which is NOT a fixed point

# convert to numpy arrays
fps_all = np.array(fps_all)
fps_threshold = np.array(fps_threshold)
fps_mutated = np.array(fps_mutated)

In [8]:
# FIGURE FOR ECC 2024

all_ecc2024_figure = bkplot.figure(
    frame_width=200,
    frame_height=150,
    x_axis_label="ps (switch protein conc.)",
    y_axis_label='Fs (switch gene tranc. reg. func.)',
    x_range=(0, max(p_switch_axis)),
    y_range=(0, 1),
    title="Burdensome gene present",
    tools='pan,box_zoom,reset,save'
)
threshold_ecc2024_figure = bkplot.figure(
    frame_width=200,
    frame_height=150,
    x_axis_label="ps (switch protein conc.)",
    y_axis_label='Fs (switch gene tranc. reg. func.)',
    x_range=(0, max(p_switch_axis)),
    y_range=(0, 1),
    title="Threshold burden",
    tools='pan,box_zoom,reset,save'
)
mutated_ecc2024_figure = bkplot.figure(
    frame_width=200,
    frame_height=150,
    x_axis_label="ps (switch protein conc.)",
    y_axis_label='Fs (switch gene tranc. reg. func.)',
    x_range=(0, max(p_switch_axis)),
    y_range=(0, 1),
    title="Burdensome gene mutated",
    tools='pan,box_zoom,reset,save'
)
figures=[all_ecc2024_figure,threshold_ecc2024_figure,mutated_ecc2024_figure]
F_reqs=[F_reqs_all,F_reqs_threshold,F_reqs_mutated]
F_req_dashes=['dashed','dashdot','solid']
fps=[fps_all,fps_threshold,fps_mutated]
for i in range(0,3):
    # svg backend
    figures[i].output_backend = "svg"

    # plot F values
    figures[i].line(p_switch_axis, F_reals, line_width=2, color=bkRGB(72, 209, 204), legend_label='real')
    figures[i].line(p_switch_axis, F_reqs[i], line_width=2, color=bkRGB(0,0,0), line_dash=F_req_dashes[i],
                    legend_label='required')

    # legend settings
    figures[i].legend.location = "top_left"
    figures[i].legend.label_text_font_size = '8pt'
    figures[i].legend.glyph_width = 15
    figures[i].legend.padding = 2
    figures[i].legend.spacing = 2
    figures[i].legend.location = 'top_left'
    figures[i].legend.margin = 5

    # handle axis ticks
    figures[i].yaxis.ticker = bkmodels.FixedTicker(ticks=[par['baseline_switch'],1])
    figures[i].yaxis.major_label_overrides = {par['baseline_switch']:'Fsb',1:'1'}
    # figures[i].xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    # figures[i].xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    # figures[i].yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    # figures[i].yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    figures[i].xaxis.major_label_text_font_size = '8pt'  # turn off x-axis tick labels
    figures[i].yaxis.major_label_text_font_size = '8pt'  # turn off y-axis tick labels
    figures[i].xaxis.ticker = bkmodels.FixedTicker(ticks=[0]+fps[i][:,0])
    figures[i].xaxis.major_label_overrides = {fps[i][j, 0]: '' for j in range(0, len(fps[i]))}

    # axis label and title font size
    figures[i].xaxis.axis_label_text_font_size = '8pt'
    figures[i].yaxis.axis_label_text_font_size = '8pt'
    figures[i].title.text_font_size = '8pt'

# mark the fixed points for all synthetic genes present
all_ecc2024_figure.scatter(fps_all[0, 0], fps_all[0, 1],
                   marker='circle',
                   size=8, line_width=2, line_color=bkRGB(222, 49, 99), fill_color=bkRGB(222, 49, 99))
all_ecc2024_figure.scatter(fps_all[1, 0], fps_all[1, 1],
                   marker='circle',
                   size=8, line_width=2, line_color=bkRGB(222, 49, 99), fill_color=bkRGB(222, 49, 99, 0))
all_ecc2024_figure.scatter(fps_all[2, 0], fps_all[2, 1],
                   marker='circle',
                   size=8, line_width=2, line_color=bkRGB(222, 49, 99), fill_color=bkRGB(222, 49, 99))
# add arrows indicating the stability of points for all synthetic genes present
arrow_length=125
arrows_start = 0
arrows_end = fps_all[0, 0]
arrows_range = np.arange(arrows_start, arrows_end+arrow_length, arrow_length)
arrows_range[-1] = arrows_end
for i in range(0, len(arrows_range)-1):
    all_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                 line_color=bkRGB(200,200,200),
                                                                 size=7),
                                         line_color=bkRGB(200,200,200), line_width=2,
                                         x_start=arrows_range[i], x_end=arrows_range[i+1],
                                         y_start=0.05, y_end=0.05))
arrows_start = fps_all[0, 0]
arrows_end = fps_all[1, 0]
arrows_range = np.arange(arrows_start, arrows_end+arrow_length, arrow_length)
arrows_range[-1] = arrows_end
for i in range(0, len(arrows_range) - 1):
    all_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                 line_color=bkRGB(200,200,200),
                                                                 size=7),
                                         line_color=bkRGB(200,200,200), line_width=2,
                                         x_start=arrows_range[i+1], x_end=arrows_range[i],
                                         y_start=0.05, y_end=0.05))
arrows_start = fps_all[1, 0]
arrows_end = fps_all[2, 0]
arrows_range = np.arange(arrows_start, arrows_end+arrow_length, arrow_length)
arrows_range[-1] = arrows_end
for i in range(0, len(arrows_range) - 1):
    all_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                 line_color=bkRGB(200,200,200),
                                                                 size=7),
                                         line_color=bkRGB(200,200,200), line_width=2,
                                         x_start=arrows_range[i], x_end=arrows_range[i+1],
                                         y_start=0.05, y_end=0.05))
arrows_start = fps_all[2, 0]
arrows_end = p_switch_sup*1.2
arrows_range = np.arange(arrows_start, arrows_end+arrow_length, arrow_length)
arrows_range[-1] = arrows_end
for i in range(0, len(arrows_range) - 1):
    all_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                 line_color=bkRGB(200,200,200),
                                                                 size=7),
                                         line_color=bkRGB(200,200,200), line_width=2,
                                         x_start=arrows_range[i + 1], x_end=arrows_range[i],
                                         y_start=0.05, y_end=0.05))

# mark the fixed points for threshold burden
threshold_ecc2024_figure.scatter(fps_threshold[0, 0], fps_threshold[0, 1],
                         marker='circle',
                         size=8, line_width=2, line_color=bkRGB(222, 49, 99), fill_color=bkRGB(222, 49, 99, 0))
threshold_ecc2024_figure.scatter(fps_threshold[1, 0], fps_threshold[1, 1],
                         marker='circle',
                         size=8, line_width=2, line_color=bkRGB(222, 49, 99), fill_color=bkRGB(222, 49, 99))
# add arrows indicating the stability of points for threshold burden
arrows_start=0
arrows_end=fps_threshold[0, 0]
arrows_range=np.arange(arrows_start, arrows_end+arrow_length, arrow_length)
arrows_range[-1] = arrows_end
for i in range(0, len(arrows_range)-1):
    threshold_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                       line_color=bkRGB(200,200,200),
                                                                       size=7),
                                               line_color=bkRGB(200,200,200), line_width=2,
                                               x_start=arrows_range[i], x_end=arrows_range[i+1],
                                               y_start=0.05, y_end=0.05))
arrows_start = fps_threshold[0, 0]
arrows_end = fps_threshold[1,0]
arrows_range = np.arange(arrows_start, arrows_end+arrow_length, arrow_length)
arrows_range[-1] = arrows_end
for i in range(0, len(arrows_range) - 1):
    threshold_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                       line_color=bkRGB(200,200,200),
                                                                       size=7),
                                               line_color=bkRGB(200,200,200), line_width=2,
                                               x_start=arrows_range[i], x_end=arrows_range[i+1],
                                               y_start=0.05, y_end=0.05))
arrows_start = fps_threshold[1,0]
arrows_end = p_switch_sup*1.2
arrows_range = np.arange(arrows_start, arrows_end+arrow_length, arrow_length)
arrows_range[-1] = arrows_end
for i in range(0, len(arrows_range) - 1):
    threshold_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                       line_color=bkRGB(200,200,200),
                                                                       size=7),
                                               line_color=bkRGB(200,200,200), line_width=2,
                                               x_start=arrows_range[i + 1], x_end=arrows_range[i],
                                               y_start=0.05, y_end=0.05))
# mark the fixed points for gene expression burden after mutation
mutated_ecc2024_figure.scatter(fps_mutated[0, 0], fps_mutated[0, 1],
                               marker='circle',
                               size=8, line_width=2, line_color=bkRGB(222, 49, 99), fill_color=bkRGB(222, 49, 99))
# add arrows indicating the stability of points for gene expression burden after mutation
arrows_start = 0
arrows_end = fps_mutated[0, 0]
arrows_range = np.arange(arrows_start,arrows_end+arrow_length,arrow_length)
arrows_range[-1] = arrows_end
for i in range(0,len(arrows_range)-1):
    mutated_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                          line_color=bkRGB(200,200,200),
                                                                            size=7),
                                                    line_color=bkRGB(200,200,200), line_width=2,
                                                    x_start=arrows_range[i], x_end=arrows_range[i+1],
                                                    y_start=0.05, y_end=0.05))
arrows_start = fps_mutated[0, 0]
arrows_end = p_switch_sup*1.2
arrows_range = np.arange(arrows_start, arrows_end+arrow_length, arrow_length)
arrows_range[-1] = arrows_end
for i in range(0, len(arrows_range) - 1):
    mutated_ecc2024_figure.add_layout(bkmodels.Arrow(end=bkmodels.NormalHead(fill_color=bkRGB(200,200,200),
                                                                        line_color=bkRGB(200,200,200),
                                                                        size=7),
                                                line_color=bkRGB(200,200,200), line_width=2,
                                                x_start=arrows_range[i+1], x_end=arrows_range[i],
                                                y_start=0.05, y_end=0.05))

# show
bkplot.show(bklayouts.column([all_ecc2024_figure, threshold_ecc2024_figure, mutated_ecc2024_figure]))
